In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dir = "part_3/"

In [2]:
PDC_closed = nx.read_weighted_edgelist("part_2/G_P_closed.nx",nodetype=int, create_using=nx.DiGraph())
PDC_open = nx.read_weighted_edgelist("part_2/G_P_open.nx",nodetype=int, create_using=nx.DiGraph())
DFT_closed = nx.read_weighted_edgelist("part_2/G_D_closed.nx",nodetype=int, create_using=nx.DiGraph())
DFT_open = nx.read_weighted_edgelist("part_2/G_D_open.nx",nodetype=int, create_using=nx.DiGraph())

In [20]:
def gen_subgraph_connected(g):
    nodes_weakly_connected = max(nx.weakly_connected_components(g), key=len)
    return PDC_closed.subgraph(list(nodes_weakly_connected))


def network_info(g):
    c = nx.average_clustering(g.to_undirected())
    p = nx.average_shortest_path_length(g)
    return (round(c/2, 2), round(p, 2))

In [21]:
def add_feature(a, header):
    df = pd.DataFrame.from_dict(a, orient="index")
    df.columns = [header]
    return df

def nodes_info(g):
    
    df_f1 = pd.read_csv("part_2/" + "S040R01.csv")
    nodes = np.arange(len(list(df_f1.columns)))
    labels_number_to_channel = dict(zip(nodes, list(df_f1.columns)))
    
    a = (dict(g.in_degree()), "In degree")
    b = (dict(g.out_degree()), "Out degree")
    c = (dict(g.degree()), "Degree")
    
    dfs = []
    for i in [a, b, c]:
        dfs.append(add_feature(i[0], i[1]))
    df = pd.concat(dfs, axis=1)
    channels = [labels_number_to_channel[k] for k in df.index]
    df['Channel'] = channels
    cols = df.columns.tolist()
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    return df.sort_values(by="Degree", ascending=False)

In [22]:
PDC_closed2 = gen_subgraph_connected(PDC_closed)

In [23]:
# clustering and path
graphs_mod = [PDC_closed, PDC_open, DFT_closed, DFT_open]
graphs = [PDC_closed, PDC_open, DFT_closed, DFT_open]

networks = ["PDC C", "PDC O", "DFT C", "DFT O"]
network_info_results = []
nodes_info_results = []

for i in graphs_mod:
    network_info_results.append(network_info(i))
for i in graphs:
    nodes_info_results.append(nodes_info(i))

In [24]:
#Create tables 3.1
# global
df_results_global = {networks[i]: network_info_results[i] for i in range(len(networks))}
df_results_global = pd.DataFrame.from_dict(df_results_global, orient="index")
df_results_global.columns = ["Avg. clustering coef.", "Ave. path length"]
df_results_global.index.rename("Graph", inplace=True)
df_results_global.to_csv(dir + "table_31_global.csv")

In [8]:
# local
fields = ["Degree", "In degree", "Out degree"]
for i in range(len(networks)):
    for field in fields:
        df_local = nodes_info_results[i][["Channel", field]].sort_values(by=field, ascending=False).head(n=10)
        df_local.to_csv(dir + "table_31_local_" + networks[i].replace(" ", "_")+ "_" + field.replace(" ", "") + ".csv", index=False)

In [9]:
f, axarr = plt.subplots(2, 2)
plt.tight_layout()
axarr[0, 0].hist(nodes_info_results[0]["Degree"].values,edgecolor = "black", density=True)
axarr[0, 0].set_title("PDC C")
axarr[0, 0].set_xlim([0,67])
axarr[0, 1].hist(nodes_info_results[1]["Degree"].values,edgecolor = "black", density=True)
axarr[0, 1].set_title("PDC O")
axarr[1, 0].hist(nodes_info_results[2]["Degree"].values,edgecolor = "black", density=True)
axarr[1, 0].set_title("DFT C")
axarr[1, 1].hist(nodes_info_results[3]["Degree"].values,edgecolor = "black", density=True)
axarr[1, 1].set_title("DFT O")
axarr[1, 0].set_xlabel("Degree")
axarr[1, 0].set_ylabel("Density")
plt.savefig(dir + "hist_degree.png")
plt.show()

AttributeError: Unknown property density

In [53]:
def mean_std(v):
    print(str(round(np.median(v),0)) + " +- " + str(round(np.std(v),0)))

a1 = [k for k in nodes_info(DFT_closed)["In degree"].values]
a2 = [k for k in nodes_info(DFT_open)["In degree"].values]
a = a1 + a2
mean_std(a)

2.0 +- 20.0


In [54]:
a1 = [k for k in nodes_info(DFT_closed)["Out degree"].values]
a2 = [k for k in nodes_info(DFT_open)["Out degree"].values]
a = a1+a2
mean_std(a)

12.0 +- 3.0


In [47]:
a1 = [k for k in nodes_info(DFT_closed)["Degree"].values]
a2 = [k for k in nodes_info(DFT_open)["Degree"].values]
a = a1+a2

b1 = [k for k in nodes_info(PDC_closed)["Degree"].values]
b2 = [k for k in nodes_info(PDC_open)["Degree"].values]
b = b1+b2

In [48]:
mean_std(a)

18.0 +- 20.0


In [49]:
mean_std(b)

22.0 +- 16.0


In [42]:
from scipy.stats import mannwhitneyu
mannwhitneyu(b1, b2, alternative="two-sided")

MannwhitneyuResult(statistic=2097.5, pvalue=0.81527942172324652)